### *Podemos rodar às segundas-feiras já que as partidas da liga são jogadas aos domingos*

In [1]:
import pandas as pd
from functions import clube_id_e_sessao, pega_dados, conecta_ao_db

In [2]:
sessao, club_id= clube_id_e_sessao()

Clube ID: 79085


### Informações gerais dos jogos

In [3]:
# Conecta ao banco de dados
db= conecta_ao_db('dados_sokker.db')
curs= db.cursor()

#Pega informações únicas das partidas já adicionadas para que não sejam inseridos valores duplicados na base
matches = pd.read_sql_query("SELECT * FROM matches", db)

ids= matches['matchID'].unique()
data_esperada= matches['dateExpected'].unique()
data_iniciada= matches['dateStarted'].unique()

In [4]:
# Id do clube a serem pegas as informações
id_clube= club_id

# Pegando informações das partidas retirando aquelas que já foram pegas
url= f'https://sokker.org/xml/matches-team-{club_id}.xml'
df_matches= pega_dados(url, sessao, search= 'match')

#Retirando valores nulos (partidas ainda não jogadas) e duplicados (partidas já adicionadas na base)
df_matches.dropna(inplace= True)

query= 'matchID not in @ids and dateExpected not in @data_esperada and dateStarted not in @data_iniciada'

# Retirando colunas que não salvaremos as informações
df_matches.drop(columns= ['homeTeamName', 'awayTeamName'], 
                inplace= True)

#df_matches= pd.concat([matches, df_matches])

df_matches= df_matches.query(query)
df_matches

,matchID,homeTeamID,awayTeamID,leagueID,round,season,week,day,dateExpected,dateStarted,homeTeamScore,awayTeamScore,supporters,weather,isFinished
33,42311075,90579,79085,51530,0,0,1124,5,2025-01-23 22:00,2025-01-23 22:00,1,0,3475,2,1
34,42310349,73989,79085,3,0,0,1124,6,2025-01-24 19:14,2025-01-24 19:14,0,0,6931,6,1
35,42311659,79085,151507,51530,0,0,1125,0,2025-01-25 22:00,2025-01-25 22:00,5,0,1244,5,1
36,42084970,79085,10365,1349,11,69,1125,1,2025-01-26 21:59,2025-01-26 21:59,5,1,10707,7,1
37,42311860,79085,62758,51530,0,0,1125,2,2025-01-27 22:00,2025-01-27 22:00,1,3,3008,6,1
38,42314656,79085,123788,51530,0,0,1125,3,2025-01-28 22:00,2025-01-28 22:00,5,1,3144,6,1
39,42084971,10365,79085,1349,12,69,1125,4,2025-01-29 21:59,2025-01-29 21:59,1,5,3100,6,1


In [5]:
if len(df_matches) > 0:
    
    # Adiciona dados à tabela
    df_matches.to_sql('matches', con=db, if_exists='append', index= False)

In [6]:
#df_matches = pd.read_sql_query("SELECT * FROM matches", db)
len(df_matches)

7

### Estatísticas dos jogos

In [7]:
#Pega informações únicas das estatísticas já adicionadas para que não sejam inseridos valores duplicados na base
stats = pd.read_sql_query("SELECT * FROM matchStats", db)

ids= stats['matchID'].unique()
team= stats['teamID'].unique()
terco_defensivo= stats['timeOnHalf'].unique()
posse_de_bola= stats['timePossession'].unique()

In [8]:
#lista que guardará as estatísticas de cada partida
list_stats= list()

# Pegando estatísticas

for ID in df_matches['matchID'].unique():
    
    url= f'https://sokker.org/xml/match-{ID}.xml'
    df= pega_dados(url, sessao, search= 'teamStats')
    df['matchID']= ID
    list_stats.append(df)
    


df_stats= pd.concat([stats for stats in list_stats])

# Retirando valores nulos (partidas ainda não jogadas) e duplicados (partidas já adicionadas na base)

df_stats.dropna(inplace= True)

query= 'matchID not in @ids and teamID not in @team and timeOnHalf not in @terco_defensivo and timePossession not in @posse_de_bola'

df_stats= df_stats.query(query)

df_stats

,teamID,timeOnHalf,timePossession,offsides,shoots,fouls,yellowCards,redCards,tacticName,ratingScoring,ratingPassing,ratingDefending,matchID
0,90579,22577,38179,0,14,2,1,0,5 - 4 - D,11,11,12,42311075
1,79085,47457,31855,0,3,6,2,0,4-3-3aRU,8,10,11,42311075
0,73989,24017,35247,3,16,1,0,0,GOOO,7,10,7,42310349
1,79085,45942,34712,3,1,1,0,0,4-3-3a,5,7,7,42310349
0,79085,20973,39695,0,27,2,0,0,4-2-3-1a,9,7,5,42311659
1,151507,49493,30771,0,0,2,0,0,Attack,5,4,4,42311659
0,79085,19904,41613,0,25,0,0,0,3-5-2-aSR,9,9,10,42084970
1,10365,50192,28483,0,3,2,0,0,Sinop,7,6,7,42084970
0,79085,45446,34145,2,4,2,1,0,3-5-2-aSR,9,9,10,42311860
1,62758,24951,36252,0,15,2,0,0,4-3-1-2,10,9,8,42311860


In [9]:
# Adicionando informações se houver alguma nova
if len(df_stats) > 0:
    
    # Adiciona dados à tabela
    df_stats.to_sql('matchStats', con=db, if_exists='append', index= False)

In [10]:
# Conferindo
stats = pd.read_sql_query("SELECT * FROM matchStats", db)
len(stats)

1532

### Estatísticas da Liga

In [11]:
#Pega informações ligas já adicionadas para que não sejam inseridos valores duplicados na base
league_stats = pd.read_sql_query("SELECT * FROM leagueStats", db)

ids= league_stats['leagueID'].unique()
team= league_stats['teamID'].unique()
rodada=  league_stats['round'].unique()

In [12]:
#Pegar o id da liga desejada entrando na liga na página do sokker
league_id= 1349

url= f'https://sokker.org/xml/league-{league_id}.xml'
df_league_stats= pega_dados(url, sessao, search= 'teams', tree_top= True)

In [13]:
df_league_stats['leagueID']= league_id
df_league_stats

,teamID,round,points,wins,draws,losses,goalsScored,goalsLost,rankTotal,leagueID
0,47841,12,33,11,0,1,49,12,33537049011010,1349
1,6295,12,30,10,0,2,66,22,30544066010009,1349
2,79866,12,26,8,2,2,32,12,26520032008005,1349
3,23755,12,23,7,2,3,56,18,23538056007011,1349
4,78671,12,21,7,0,5,44,19,21525044007007,1349
5,79085,12,19,5,4,3,30,13,19517030005006,1349
6,89569,12,19,6,1,5,27,27,19500027006002,1349
7,124139,12,15,5,0,7,31,28,15503031005008,1349
8,122169,12,13,4,1,7,52,42,13510052004004,1349
9,48541,12,9,3,0,9,16,73,9443016003001,1349


In [14]:
# Adicionando informações 
if len(df_league_stats) > 0:
    
    # Adiciona dados à tabela
    df_league_stats.to_sql('leagueStats', con=db, if_exists='append', index= False)

league_stats = pd.read_sql_query("SELECT * FROM leagueStats", db)
len(league_stats)

### Liga informação geral

In [15]:
#Pega informações das ligas já adicionadas para que não sejam inseridos valores duplicados na base
leagues = pd.read_sql_query("SELECT * FROM league", db)

ids= leagues['leagueID'].unique()
league_name= leagues['name'].unique()

In [16]:
#Pegando dados da liga
df_league= pega_dados(url, sessao, search= 'info')

#Retirando coluna que não será utilizada
df_league.drop(columns= ['userID', 'round'], inplace= True)

#Retirando valores duplicados (se as estatísticas já tiverem sido adicionadas na base)
query= 'leagueID not in @ids and name not in @league_name'
df_league= df_league.query(query)

df_league

,leagueID,name,countryID,division,season,type,isOfficial,isCup


In [17]:
# Adicionando informações se for uma liga ainda não adicionada
if len(df_league) > 0:
    
    # Adiciona dados à tabela
    df_league.to_sql('league', con=db, if_exists='append', index= False)

In [18]:
league = pd.read_sql_query("SELECT * FROM league", db)
len(league)

4

In [19]:
# Salvando alterações
db.commit()

# Fechando conexão com a base de dados
db.close()